# Import regions
This notebook creates a table for holding polygon shapes. These shapes can then be used to generate statistics in regions specified by those polygons.

## Prerequisites
This notebook assumes that you have created a PostgreSQL database with the PostGIS extension installed:
- `CREATE DATABASE socat_kpi;`
- `CREATE EXTENSION postgis;`

## Setup
Imports, constants etc.

In [1]:
import psycopg2
from osgeo import ogr

DB_HOST = 'localhost'
DB_USER = 'postgres'
DB_PASSWORD = 'postgres'
DB_NAME = 'socat_kpi'

## Connect to database and Clean
Connect to the database and delete any existing tables.

In [2]:
conn = psycopg2.connect(database = DB_NAME, 
                        user = DB_USER, 
                        host= DB_HOST,
                        password = DB_PASSWORD)

cur = conn.cursor()

cur.execute("DROP TABLE IF EXISTS regions")
conn.commit()

## Create the Regions table
This table will contain the following fields:

| Field    | Description                        |
|----------|------------------------------------|
| `source` | The entity that defined the region |
| `name`   | The name of the region             |
| `shape`  | The polygon defining the region    |

Each record in the table will contain a single polygon.

In [3]:
cur.execute("""CREATE TABLE regions(
id bigserial primary key,
source text,
name text,
shape geometry(Polygon, 4326)
);""")
conn.commit()

## Import single-polygon files
This cell can be used to import files containing single polygons.

Specify the list of files in `POLY_FILES`, which is an array of arrays. Each sub-array should contain `[filename, source, name]`.

The import logic is adapted from [Importing shapefile to PostGIS using Python and ogr](https://gis.stackexchange.com/questions/90085/importing-shapefile-to-postgis-using-python-and-ogr).

In [4]:
POLY_FILES = [
    ['shapes/ospar_regions/ArcticWaters.shp', 'OSPAR', 'Arctic Waters'],
    ['shapes/ospar_regions/GreaterNorthSea.shp', 'OSPAR', 'Greater North Sea'],
    ['shapes/ospar_regions/CelticSeas.shp', 'OSPAR', 'Celtic Seas'],
    ['shapes/ospar_regions/BiscayIberia.shp', 'OSPAR', 'Bay of Biscay and Iberian Coast'],
    ['shapes/ospar_regions/WiderAtlantic.shp', 'OSPAR', 'Wider Atlantic'],
    ['shapes/my_regions/Baltic.shp', 'N/A', 'Baltic'],
    ['shapes/my_regions/Mediterranean.shp', 'N/A', 'Mediterranean']
]

for p in POLY_FILES:
    shapefile = ogr.Open(p[0])    
    layer = shapefile.GetLayer(0)    
    feature = layer.GetFeature(0)
    wkt = feature.GetGeometryRef().ExportToWkt()
    cur.execute(f"INSERT INTO regions (source, name, shape) VALUES ('{p[1]}', '{p[2]}', ST_GeometryFromText('{wkt}', 4326))")
    conn.commit()


/home/steve.jones@vliz.be/Documents/JPI_KHUB/socat_kpis/venv/lib/python3.12/site-packages/osgeo/ogr.py:593: FutureWarning: Neither ogr.UseExceptions() nor ogr.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


## CCAMLR Regions
Another example of regions around the Southern Ocean. One of the entries in this shapefile is a MULTIPOLYGON. For now I'm just ignoring it.

In [5]:
shapefile = ogr.Open('shapes/CCAMLR/geographical_data/asd/CCAMLR_ASD_EPSG4326.shp')
layer = shapefile.GetLayer(0)

feature = layer.GetNextFeature()
while feature is not None:
    geometry = feature.GetGeometryRef()
    if geometry.GetGeometryType() == ogr.wkbPolygon:
        wkt = feature.GetGeometryRef().ExportToWkt()
        cur.execute(f"INSERT INTO regions (source, name, shape) VALUES ('CCAMLR', '{feature.GAR_Name}', ST_GeometryFromText('{wkt}', 4326))")
    feature = layer.GetNextFeature()
conn.commit()


## Create Index
Speed is king

In [6]:
cur.execute('CREATE INDEX regions_idx ON regions USING GIST(shape)')
conn.commit()

## Close everything down

In [7]:
cur.close()
conn.close()